In [2]:
import numpy as np
import pandas as pd

In [ ]:
# 1. Read the two CSV files into separate DataFrames
df_us = pd.read_csv('../output-data/us_names_with_popularity_and_connotations_cleaned.csv')
df_uk = pd.read_csv('../output-data/uk_names_with_popularity_and_connotations_cleaned.csv')

# 2. Combine both DataFrames
df_combined = pd.concat([df_us, df_uk], ignore_index=True)

# 3. For each name, sum up 'n' (similar to ave(..., FUN = sum))
df_combined['n'] = df_combined.groupby('lowercase_name')['n'].transform('sum')

# 4. Remove duplicated names, keeping the first occurrence
df_combined = df_combined.drop_duplicates(subset='lowercase_name', keep='first')



In [4]:
df_combined.head()

,lowercase_name,name,sex,n,year,per_year,percent_per_year,nchar,flag,intelligence,...,religious,tradition,connotation_1,connotation_2,connotation_3,connotation_4,connotation_5,count,connotation_raw,note
0,aaban,Aaban,M,256.0,2022,1741954,0.000402,5,False,False,...,False,True,dignity,nobility,prosperity,leadership,strength,NaN,NaN,NaN
13,aabha,Aabha,F,65.0,2019,1676568,0.000358,5,False,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,aabid,Aabid,M,68.0,2003,1975097,0.000253,5,NaN,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,aabidah,Aabidah,F,19.0,2018,1699569,0.000294,7,NaN,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,aabir,Aabir,M,19.0,2022,1741954,0.000517,5,NaN,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Select only the specified columns
columns_to_keep = ['name', 'sex', 'year', 'n', 'percent_per_year', 
                   'connotation_1', 'connotation_2', 'connotation_3', 
                   'connotation_4', 'connotation_5']

# Create a new dataframe with only the selected columns
filtered_data = df_combined[columns_to_keep]

# Display the first few rows to verify
filtered_data.head()


,name,sex,year,n,percent_per_year,connotation_1,connotation_2,connotation_3,connotation_4,connotation_5
0,Aaban,M,2022,256.0,0.000402,dignity,nobility,prosperity,leadership,strength
13,Aabha,F,2019,65.0,0.000358,NaN,NaN,NaN,NaN,NaN
23,Aabid,M,2003,68.0,0.000253,NaN,NaN,NaN,NaN,NaN
26,Aabidah,F,2018,19.0,0.000294,NaN,NaN,NaN,NaN,NaN
27,Aabir,M,2022,19.0,0.000517,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop rows where connotation_1 is NaN
filtered_data_no_na = filtered_data.dropna(subset=['connotation_1'])

# Display the first few rows to verify the NaN values have been removed
filtered_data_no_na.head()

,name,sex,year,n,percent_per_year,connotation_1,connotation_2,connotation_3,connotation_4,connotation_5
0,Aaban,M,2022,256.0,0.000402,dignity,nobility,prosperity,leadership,strength
40,Aadam,M,1995,2638.0,0.000315,first human,earth,life,creation,origin
72,Aadan,M,2003,139.0,0.000253,spiritual,strong,traditional,wise,noble
84,Aadarsh,M,2001,266.0,0.000360,ideal,perfect,model,exemplary,principle
111,Aaden,M,2020,5109.0,0.003249,fire,warmth,light,passion,strength


In [7]:
# Write the filtered data to a CSV file
output_path = '../output-data/recent-us-and-uk-names.csv'
filtered_data_no_na.to_csv(output_path, index=False)


That gives us a CSV, but we can do better than this and create a file that has some connections...

In [8]:
# Create a SQLite database with three tables: names, connotations, and connections

import sqlite3
import pandas as pd

# Create a connection to the SQLite database
db_path = '../output-data/us-and-uk-names.db'
conn = sqlite3.connect(db_path)

# Create the names table
conn.execute('''
CREATE TABLE IF NOT EXISTS names (
    name TEXT PRIMARY KEY,
    sex TEXT,
    n INTEGER,
    percent_per_year REAL
)
''')

# Create the connotations table
conn.execute('''
CREATE TABLE IF NOT EXISTS connotations (
    connotation TEXT PRIMARY KEY
)
''')

# Create the connections table
conn.execute('''
CREATE TABLE IF NOT EXISTS connections (
    name TEXT,
    connotation TEXT,
    FOREIGN KEY (name) REFERENCES names (name),
    FOREIGN KEY (connotation) REFERENCES connotations (connotation),
    PRIMARY KEY (name, connotation)
)
''')

# Insert data into the names table
names_data = filtered_data_no_na[['name', 'sex', 'n', 'percent_per_year']]
names_data.to_sql('names', conn, if_exists='replace', index=False)

# Extract all unique connotations
all_connotations = set()
for col in ['connotation_1', 'connotation_2', 'connotation_3', 'connotation_4', 'connotation_5']:
    all_connotations.update(filtered_data_no_na[col].dropna().unique())

# Insert connotations into the connotations table
connotations_df = pd.DataFrame({'connotation': list(all_connotations)})
connotations_df.to_sql('connotations', conn, if_exists='replace', index=False)

# Create connections data
connections_data = []
for _, row in filtered_data_no_na.iterrows():
    name = row['name']
    for col in ['connotation_1', 'connotation_2', 'connotation_3', 'connotation_4', 'connotation_5']:
        if pd.notna(row[col]):
            connections_data.append({'name': name, 'connotation': row[col]})

# Insert connections into the connections table
connections_df = pd.DataFrame(connections_data)
connections_df.to_sql('connections', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"SQLite database created at {db_path} with tables: names, connotations, and connections")


SQLite database created at ../output-data/us-and-uk-names.db with tables: names, connotations, and connections
